In [ ]:
source('helpers.R')

In [3]:
prepost <- readRDS("../../../ProcessedData/seurat/annotated_prepost_csf.rds")

In [4]:
target <- subset(prepost, cell_type %in%  c('CD4+ T', 'CD4+ CAR T', 'CD8+ T', 'CD8+ CAR T', 'Cycling CD8+ T')) 

target$cluster = 'CD8'
target$cluster[target$cell_type %in% c('CD4+ T', 'CD4+ CAR T')] = 'CD4'

target$group_by_cluster = 'CART' 
target$group_by_cluster[target$cell_type %!in% c('CD4+ CAR T', 'CD8+ CAR T')] = 'Endogenous'

target$group_by_transcript = 'CART' 
target$group_by_transcript[target$CAR_Transcript == 'CAR -'] = 'Endogenous'

target$sample = paste0(target$sample_id, '_', target$group_by_cluster)

In [ ]:
muscat.edgeR <- muscat.de.genes(
    seurat_obj = target, 
    cluster_col = 'cluster', 
    group_col = 'group_by_cluster', 
    sample_col = 'sample', 
    method = 'edgeR'
)

protein_coding <- filter_genes_by_biotype(muscat.edgeR$gene, biotype = "protein_coding")
muscat.edgeR <- muscat.edgeR[muscat.edgeR$gene %in%  c(protein_coding$filtered_genes, 'CAR'),]

In [ ]:
cd4.genes.up    <- c("CAR","TNFRSF9","TNFSF4","EBI3","IRAK3","ZC3H12C","GZMB","CCL4","CXCL13","XCL1","KLRD1","HLA-DQA1","DAPK2","ITGA1","NFIL3")
cd4.genes.down  <- c("TCF7","CCR6","CXCR5","NR3C2","KLF3","WNT7A","GREM2","MYBL1","PLCB1","CLCF1","PLXNA4","AFF3","ZFYVE9","FNBP1L","SEMA5A")
cd8.genes.up    <- c("CAR","CXCL13","PLS3","CDC42BPA","CTHRC1","TLE1","RUNX2","DOCK5","CAPG","SPINK2","XCL1","RIMS2","RBPJ","KLRC1","SPRY1")
cd8.genes.down  <- c("S1PR5","TLR3","KLF3","CX3CR1","CXCR5","DKK3","CLCF1","MYBL1","GPR183","TYROBP","TGFBR3","DUSP8","PDGFD","KLRG1","CADM1")

In [ ]:
ctype = "CD8"
muscat.edgeR.ctype <- muscat.edgeR %>% 
    filter(cluster_id == ctype) %>% 
    dplyr::select(gene, p_adj.glb, logFC)

rownames(muscat.edgeR.ctype) <- muscat.edgeR.ctype$gene
colnames(muscat.edgeR.ctype) <- c('gene', 'p_val_adj', 'avg_log2FC')

muscat.edgeR.ctype <- muscat.edgeR.ctype %>% dplyr::select(p_val_adj, avg_log2FC)
muscat.edgeR.ctype$avg_log2FC <- muscat.edgeR.ctype$avg_log2FC*-1

options(repr.plot.width=4, repr.plot.height=4, repr.plot.res=200)
p <- do_VolcanoPlot(sample = target, genes.up = cd8.genes.up, genes.down = cd8.genes.down,
            de_genes = muscat.edgeR.ctype, pt.size=0.6,
            pval_cutoff = 0.05,
            FC_cutoff = log(2), n_genes = 10, order_tags_by='custom') 
p

In [ ]:
pdf(file.path(panel.path, paste0("1F_",ctype,".pdf")), width=4, height=4)
p
dev.off()

In [ ]:
ctype = "CD4"
muscat.edgeR.ctype <- muscat.edgeR %>% 
    filter(cluster_id == ctype) %>% 
    dplyr::select(gene, p_adj.glb, logFC)

rownames(muscat.edgeR.ctype) <- muscat.edgeR.ctype$gene
colnames(muscat.edgeR.ctype) <- c('gene', 'p_val_adj', 'avg_log2FC')

muscat.edgeR.ctype <- muscat.edgeR.ctype %>% dplyr::select(p_val_adj, avg_log2FC)
muscat.edgeR.ctype$avg_log2FC <- muscat.edgeR.ctype$avg_log2FC*-1

options(repr.plot.width=4, repr.plot.height=4, repr.plot.res=200)
p <- do_VolcanoPlot(sample = target, genes.up = cd4.genes.up, genes.down = cd4.genes.down,
            de_genes = muscat.edgeR.ctype, pt.size=0.6,
            pval_cutoff = 0.05,
            FC_cutoff = log(2), n_genes = 10, order_tags_by='custom') 
p

In [ ]:
pdf(file.path(panel.path, paste0("1G_",ctype,".pdf")), width=4, height=4)
p
dev.off()